# Most Frequently Used Start/End BT Sensors By Hour
Quick solution to finding the most frequently used start/end BT sensors of the origin/destinations.

Currently using the sample table provided at `qchen.sample_od_start_end` which contains one day of sample data.

In [42]:
%matplotlib inline
from psycopg2 import connect
import psycopg2.sql as pg
import configparser
from datetime import datetime, timedelta, date
import itertools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import pandas.io.sql as pandasql
from IPython.display import HTML, display

In [43]:
# setting up pgsql connection
CONFIG = configparser.ConfigParser()
CONFIG.read('db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [48]:
# Get start sensor column names
sql = pg.SQL('''WITH columns AS (
                SELECT column_name
                  FROM information_schema.columns
                  WHERE table_schema='qchen' AND table_name='sample_od_start_end'
                )
                SELECT column_name FROM columns
                WHERE column_name SIMILAR TO 's\_%';''')
columns = pandasql.read_sql(sql, con)
len_columns = len(columns)
# Get number of observations for each start sensor
col_values = []
for i in range(len_columns):
    sql = pg.SQL('''SELECT {sensor_id} AS sensor, SUM({column_name}) AS obs, trunc(EXTRACT(hour from measuredtimetimestamp)) AS "hour"
                FROM qchen.sample_od_start_end
                GROUP BY trunc(EXTRACT(hour from measuredtimetimestamp))
                ORDER BY trunc(EXTRACT(hour from measuredtimetimestamp));''').format(sensor_id = pg.Literal(columns.values[i][0]),
                                                                            column_name = (pg.Identifier(columns.values[i][0])))
    col_hourly_sum = pandasql.read_sql(sql, con)
    col_values.append(col_hourly_sum.values)
top_by_hour = []
# gets top 10 start sensors with highest obs per hour
for hour in range(24):
    ranked_sensors = []
    for col in range(len(col_values)):
        ranked_sensors.append(col_values[col][hour])
    ranked_sensors.sort(key=lambda x: x[1], reverse=True)
    top_by_hour.append(ranked_sensors[0])
    df = pd.DataFrame(ranked_sensors[:10], columns=['Sensor', 'Observations', 'Hour'])
    display(HTML(df.to_html(index=False)))

Sensor,Observations,Hour
s_fb_sp,22,0.0
s_fb_ba,13,0.0
s_co_ba,12,0.0
s_co_un,12,0.0
s_ad_yo,12,0.0
s_du_sp,10,0.0
s_du_un,10,0.0
s_du_ja,10,0.0
s_ad_un,9,0.0
s_fr_ja,9,0.0


Sensor,Observations,Hour
s_du_ja,15,1.0
s_co_ba,13,1.0
s_fb_sp,12,1.0
s_du_un,10,1.0
s_kn_df,9,1.0
s_fr_sp,9,1.0
s_fb_yk,9,1.0
s_co_un,8,1.0
s_du_sp,8,1.0
s_ad_un,8,1.0


Sensor,Observations,Hour
s_du_ja,11,2.0
s_ad_sp,11,2.0
s_co_ba,10,2.0
s_co_un,10,2.0
s_du_df,7,2.0
s_kn_df,7,2.0
s_du_sp,6,2.0
s_du_un,6,2.0
s_fr_ja,6,2.0
s_qu_ro,5,2.0


Sensor,Observations,Hour
s_du_ja,14,3.0
s_co_ba,13,3.0
s_du_un,8,3.0
s_du_df,7,3.0
s_rm_sp,6,3.0
s_rm_pa,6,3.0
s_fr_sp,6,3.0
s_du_yo,5,3.0
s_qu_sp,5,3.0
s_rm_ba,5,3.0


Sensor,Observations,Hour
s_fb_sp,11,4.0
s_rm_pa,9,4.0
s_fr_ja,8,4.0
s_fb_ba,8,4.0
s_du_yo,7,4.0
s_fb_yk,7,4.0
s_co_un,6,4.0
s_du_df,6,4.0
s_du_un,6,4.0
s_du_pa,6,4.0


...

Sensor,Observations,Hour
s_fb_sp,59,19.0
s_fb_ba,56,19.0
s_co_un,55,19.0
s_ad_yo,52,19.0
s_du_ja,47,19.0
s_co_ba,45,19.0
s_fr_pa,43,19.0
s_fr_ja,41,19.0
s_kn_df,40,19.0
s_du_un,39,19.0


Sensor,Observations,Hour
s_du_ja,55,20.0
s_fb_sp,52,20.0
s_co_un,50,20.0
s_co_ba,45,20.0
s_ad_yo,42,20.0
s_fb_ba,42,20.0
s_du_yo,39,20.0
s_du_df,37,20.0
s_du_un,34,20.0
s_qu_un,34,20.0


Sensor,Observations,Hour
s_co_un,55,21.0
s_fb_sp,46,21.0
s_du_ja,45,21.0
s_co_ba,41,21.0
s_ad_yo,38,21.0
s_fr_ja,36,21.0
s_fr_pa,33,21.0
s_fb_ba,33,21.0
s_du_un,32,21.0
s_qu_un,29,21.0


Sensor,Observations,Hour
s_rm_pa,32,22.0
s_co_ba,31,22.0
s_co_un,29,22.0
s_du_yo,29,22.0
s_fb_sp,29,22.0
s_du_df,25,22.0
s_du_un,24,22.0
s_ad_un,21,22.0
s_fr_pa,21,22.0
s_du_ba,20,22.0


Sensor,Observations,Hour
s_co_un,33,23.0
s_fb_yk,32,23.0
s_co_ba,28,23.0
s_du_un,28,23.0
s_fb_sp,28,23.0
s_du_yo,25,23.0
s_rm_pa,23,23.0
s_du_ja,21,23.0
s_fr_un,21,23.0
s_du_df,19,23.0


In [49]:
# Overall top start nodes by hour
df = pd.DataFrame(top_by_hour, columns=['Sensor', 'Observations', 'Hour'])
display(HTML(df.to_html(index=False)))

Sensor,Observations,Hour
s_fb_sp,22,0.0
s_du_ja,15,1.0
s_du_ja,11,2.0
s_du_ja,14,3.0
s_fb_sp,11,4.0
s_fb_sp,42,5.0
s_fb_sp,70,6.0
s_rm_pa,105,7.0
s_rm_pa,123,8.0
s_fb_ba,119,9.0


In [50]:
# Get end sensor column names
sql = pg.SQL('''WITH columns AS (
                SELECT column_name
                  FROM information_schema.columns
                  WHERE table_schema='qchen' AND table_name='sample_od_start_end'
                )
                SELECT column_name FROM columns
                WHERE column_name SIMILAR TO 'e\_%';''')
columns = pandasql.read_sql(sql, con)
len_columns = len(columns)
# Get number of observations for each end sensor
col_values = []
for i in range(len_columns):
    sql = pg.SQL('''SELECT {sensor_id} AS sensor, SUM({column_name}) AS obs, trunc(EXTRACT(hour from measuredtimetimestamp)) AS "hour"
                FROM qchen.sample_od_start_end
                GROUP BY trunc(EXTRACT(hour from measuredtimetimestamp))
                ORDER BY trunc(EXTRACT(hour from measuredtimetimestamp));''').format(sensor_id = pg.Literal(columns.values[i][0]),
                                                                            column_name = (pg.Identifier(columns.values[i][0])))
    col_hourly_sum = pandasql.read_sql(sql, con)
    col_values.append(col_hourly_sum.values)
top_by_hour = []
# gets top 10 end sensors with highest obs per hour
for hour in range(24):
    ranked_sensors = []
    for col in range(len(col_values)):
        ranked_sensors.append(col_values[col][hour])
    ranked_sensors.sort(key=lambda x: x[1], reverse=True)
    top_by_hour.append(ranked_sensors[0])
    df = pd.DataFrame(ranked_sensors[:10], columns=['Sensor', 'Observations', 'Hour'])
    display(HTML(df.to_html(index=False)))

Sensor,Observations,Hour
e_fb_sp,16,0.0
e_fr_ja,15,0.0
e_co_ba,13,0.0
e_fr_sp,12,0.0
e_fr_yo,12,0.0
e_co_un,11,0.0
e_du_pa,11,0.0
e_fb_yk,11,0.0
e_qu_pa,10,0.0
e_ad_pa,10,0.0


Sensor,Observations,Hour
e_fb_sp,15,1.0
e_co_ba,12,1.0
e_co_un,10,1.0
e_du_yo,9,1.0
e_qu_sp,9,1.0
e_fr_sp,9,1.0
e_fb_ba,9,1.0
e_du_df,8,1.0
e_fr_ja,8,1.0
e_du_ro,7,1.0


Sensor,Observations,Hour
e_fb_sp,11,2.0
e_co_ba,9,2.0
e_du_yo,9,2.0
e_du_pa,9,2.0
e_fr_sp,8,2.0
e_fb_ba,8,2.0
e_fb_yk,8,2.0
e_co_un,6,2.0
e_qu_ro,6,2.0
e_qu_df,6,2.0


Sensor,Observations,Hour
e_fb_sp,15,3.0
e_du_ja,11,3.0
e_co_ba,9,3.0
e_du_df,9,3.0
e_co_un,7,3.0
e_du_yo,6,3.0
e_rm_un,6,3.0
e_fr_sp,6,3.0
e_qu_ro,5,3.0
e_qu_bv,5,3.0


Sensor,Observations,Hour
e_fb_yk,14,4.0
e_du_ja,13,4.0
e_co_un,11,4.0
e_ad_yo,8,4.0
e_co_ba,7,4.0
e_fr_un,7,4.0
e_fr_yo,7,4.0
e_fb_sp,7,4.0
e_fb_ba,6,4.0
e_du_un,5,4.0


...

Sensor,Observations,Hour
e_fb_sp,76,19.0
e_co_un,52,19.0
e_fb_yk,49,19.0
e_ea_bv,47,19.0
e_co_ba,46,19.0
e_du_ja,46,19.0
e_fr_ja,45,19.0
e_du_pa,44,19.0
e_du_df,43,19.0
e_fr_yo,43,19.0


Sensor,Observations,Hour
e_fb_ba,57,20.0
e_du_pa,53,20.0
e_du_ja,45,20.0
e_fb_sp,43,20.0
e_fb_yk,43,20.0
e_co_un,42,20.0
e_ad_pa,38,20.0
e_du_df,35,20.0
e_co_ba,32,20.0
e_fr_ja,32,20.0


Sensor,Observations,Hour
e_du_pa,46,21.0
e_fb_yk,44,21.0
e_fb_sp,43,21.0
e_fr_ja,39,21.0
e_co_ba,37,21.0
e_co_un,37,21.0
e_ea_bv,37,21.0
e_fb_ba,34,21.0
e_du_ja,33,21.0
e_ad_pa,31,21.0


Sensor,Observations,Hour
e_fb_sp,37,22.0
e_co_un,33,22.0
e_du_ja,33,22.0
e_fb_yk,33,22.0
e_co_ba,32,22.0
e_du_pa,28,22.0
e_ad_pa,24,22.0
e_du_yo,22,22.0
e_fr_un,22,22.0
e_kn_df,21,22.0


Sensor,Observations,Hour
e_fb_sp,46,23.0
e_co_un,27,23.0
e_fb_yk,27,23.0
e_fr_ja,26,23.0
e_fr_un,24,23.0
e_co_ba,21,23.0
e_du_ja,20,23.0
e_du_df,19,23.0
e_du_pa,19,23.0
e_qu_un,16,23.0


In [51]:
# Overall top end nodes by hour
df = pd.DataFrame(top_by_hour, columns=['Sensor', 'Observations', 'Hour'])
display(HTML(df.to_html(index=False)))

Sensor,Observations,Hour
e_fb_sp,16,0.0
e_fb_sp,15,1.0
e_fb_sp,11,2.0
e_fb_sp,15,3.0
e_fb_yk,14,4.0
e_du_ja,19,5.0
e_we_yo,48,6.0
e_we_yo,93,7.0
e_we_yo,126,8.0
e_co_un,84,9.0
